In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')

In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D7940531D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D794053CE0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hello, My name is Thanh and I am an AI engineer.")])

AIMessage(content="Hi Thanh, \n\nIt's nice to meet you! \n\nWhat kind of AI engineering work do you do?  I'm always interested in learning more about what people are building with AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 22, 'total_tokens': 67, 'completion_time': 0.081818182, 'prompt_time': 0.000163449, 'queue_time': 0.014165729, 'total_time': 0.081981631}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8bc4b496-82a9-4997-92e3-21c3f7ae63c3-0', usage_metadata={'input_tokens': 22, 'output_tokens': 45, 'total_tokens': 67})

In [12]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hello, My name is Thanh and I am an AI Engineer."),
    AIMessage(content="Hi Thanh, \n\nIt's nice to meet you! \n\nWhat kind of AI engineering work do you do?  I'm always interested in learning more about what people are building with AI.\n"),
    HumanMessage(content="What is my name and my job?")
])

AIMessage(content="You said your name is Thanh and you are an AI Engineer. 😊  \n\nIs there anything else you'd like to talk about?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 83, 'total_tokens': 114, 'completion_time': 0.056363636, 'prompt_time': 0.00326389, 'queue_time': 0.011276419, 'total_time': 0.059627526}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8ca3f5fc-7b91-4cee-8127-23e8a622c768-0', usage_metadata={'input_tokens': 83, 'output_tokens': 31, 'total_tokens': 114})

### Message History using Langchain

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [14]:
config = {"configurable": {"session_id":"chat1"}}

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content='Hello, My name is Thanh and I am an AI Engineer.')],
    config=config
)
response.content

"Hello Thanh! It's nice to meet you. \n\nAs an AI, I don't have a name or profession, but I'm here to help you with any questions or tasks you may have.  \n\nWhat can I do for you today? 😊  \n\n"

In [16]:
with_message_history.invoke(
    [HumanMessage(content="What is my name and my job?")],
    config=config
)

AIMessage(content='You told me your name is Thanh, and that you are an AI Engineer!  \n\nIs there anything else I can help you with? 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 97, 'total_tokens': 130, 'completion_time': 0.06, 'prompt_time': 0.003472629, 'queue_time': 0.01119491, 'total_time': 0.063472629}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-73e4820a-3678-4f9c-9804-10d0eee9e870-0', usage_metadata={'input_tokens': 97, 'output_tokens': 33, 'total_tokens': 130})

In [17]:
config1 = {"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content='Hello, My name is Thanh and I am an developer.')],
    config=config1
)
response.content

"Hello Thanh! It's nice to meet you. \n\nI'm glad to know you're also a developer. What kind of development do you specialize in?  \n\nPerhaps we could talk about some interesting projects you're working on, or technologies you're interested in.  I'm always eager to learn more about what developers are up to these days. \n\n"

In [18]:
with_message_history.invoke(
    [HumanMessage(content="What is my name and my job?")],
    config=config1
)

AIMessage(content="You told me your name is Thanh and that you are a developer.  \n\nIs there anything else you'd like to tell me about yourself or discuss? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 118, 'total_tokens': 155, 'completion_time': 0.067272727, 'prompt_time': 0.00351722, 'queue_time': 0.010809238, 'total_time': 0.070789947}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d8a66e8d-9658-41c9-b135-938640a7ff6e-0', usage_metadata={'input_tokens': 118, 'output_tokens': 37, 'total_tokens': 155})

### Prompt templates

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [20]:
chain.invoke({"messages":[HumanMessage(content="Hello, My name is Thanh.")]})

AIMessage(content="Hello Thanh, it's nice to meet you! \n\nI'm happy to help with any questions you have. What can I do for you today?  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 32, 'total_tokens': 69, 'completion_time': 0.067272727, 'prompt_time': 0.000345898, 'queue_time': 0.013659620999999999, 'total_time': 0.067618625}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-55931501-29cf-4e1e-a0c4-cbe6d0d30660-0', usage_metadata={'input_tokens': 32, 'output_tokens': 37, 'total_tokens': 69})

In [21]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [25]:
config2 = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is Thanh and I am an investor.")],
    config=config2
)

response

AIMessage(content="Hi Thanh, it's great to meet you! \n\nSo you're an investor - that's fascinating!  What kinds of investments do you typically focus on? \n\nDo you have a favorite sector or industry you enjoy working in? I'm always eager to learn more about different fields. \n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 196, 'total_tokens': 264, 'completion_time': 0.123636364, 'prompt_time': 0.005814789, 'queue_time': 0.008540210999999999, 'total_time': 0.129451153}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0fbec24d-ee5c-4637-9ab6-6aeaa1325a1a-0', usage_metadata={'input_tokens': 196, 'output_tokens': 68, 'total_tokens': 264})

In [26]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my job?")],
    config=config2
)

response.content

"You told me your name is Thanh and that you are an investor!  😄 \n\nIs there anything else you'd like to tell me about your work as an investor?  \n\n"

In [27]:
prompt_language = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of you ability in {language}"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain_language = prompt_language|model

In [32]:
response = chain_language.invoke(
    {
        "messages": [HumanMessage(content="Hi my name is Thanh")],
        "language": "German"
    }
)
response.content

'Hallo Thanh, schön dich kennenzulernen! Wie kann ich dir helfen? \n'

Wrap this more complicated chain in a Message History class.

In [34]:
with_message_history = RunnableWithMessageHistory(
    chain_language,
    get_session_history=get_session_history,
    input_messages_key="messages"
)

In [36]:
config3 = {"configurable": {"session_id": "chat4"}}
response = with_message_history.invoke(
    {'messages': [HumanMessage(content="what is my job?")], "language":"German"},
    config=config
)
response.content

'Du bist ein KI-Ingenieur!  \n\nMöchtest du noch etwas wissen? 😊  \n'

In [37]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What is my name?")], "language": "German"},
    config=config
)
response.content

'Dein Name ist Thanh. \n\nWas kann ich noch für dich tun? 😊 \n'

# Managing the conversation History

In [40]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [43]:
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [44]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

trimmed_chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt_language
    | model
    | parser
)

In [45]:
response = trimmed_chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What math problem did i ask?")],
        "language": "German",
    }
)
response

'Du hast gefragt: "Was ist 2 + 2?" \n\n\n\n'

In [50]:
# Warp this in the Message history
with_message_history = RunnableWithMessageHistory(
    trimmed_chain,
    get_session_history=get_session_history,
    input_messages_key="messages"
)
config = {"configurable": {"session_id": "chat5"}}

In [51]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What math problem did i ask?")],
        "language": "German"
    },
    config=config
)

In [52]:
response

'Du hast gefragt: Was ist 2 + 2?  \n\n\n\n'